In [226]:
from scipy.signal import argrelextrema
import pandas as pd
import numpy as np
import pandas_ta as ta

In [227]:
def isSupport(df,i):
  # fractal (i-2)>(i-1)>i<(i+1)<(i+2)
  support = df['Low'][i] < df['Low'][i-1]  and df['Low'][i] < df['Low'][i+1] \
  and df['Low'][i+1] < df['Low'][i+2] and df['Low'][i-1] < df['Low'][i-2]

  return support

def isResistance(df,i):
  resistance = df['High'][i] > df['High'][i-1]  and df['High'][i] > df['High'][i+1] \
  and df['High'][i+1] > df['High'][i+2] and df['High'][i-1] > df['High'][i-2] 

  return resistance


In [228]:
pair = "agld".upper()
tf = '4h'
data=pd.read_csv(f"C:\\Users\Grant\Desktop\work_git\work\data\\{tf}\{pair}USDT_{tf}.csv", parse_dates=True)
# data=pd.read_csv("C:\\Users\Grant\Desktop\work_git\work\data\\1d\ENSUSDT_1d.csv", parse_dates=True)
# data[-55:]

########################################
levels = []
levels_low=[]
levels_high=[]

  # start from 2 because i need i-2
  # end at shape[0]-2 because  i need i+2
  # i is the number of current row
for i in range(2,data.shape[0]-2):
    if isSupport(data,i):
        levels.append((i,data['Low'][i]))
        levels_low.append((i,data['Low'][i]))
        
    elif isResistance(data,i):
        levels.append((i,data['High'][i]))
        levels_high.append((i,data['High'][i]))
    
#############################

for i in levels:
  rowIndex = data.index[i[0]]
  data.loc[rowIndex, 'levels'] = i[1]

for i in levels_low:
  rowIndex = data.index[i[0]]
  data.loc[rowIndex, 'levels_low'] = i[1]

for i in levels_high:
  rowIndex = data.index[i[0]]
  data.loc[rowIndex, 'levels_high'] = i[1]

###########################
for ema_length in range(20,222):
    data[ema_length] = ta.ema(data["Close"], length=ema_length)

##############################
mul = 1.02

for i in range(20,222):
      # data[i] = np.where((data[i].values > data['levels'].values) & (data[i].values < ((data['levels'].values) * mul)), i, np.nan)
      data[i] = np.where((data[i].values < data['levels'].values) & (data['levels'].values  < ((data[i].values) *mul)), i, np.nan)

# for i in range(20,222):
#       data[i] = np.where((data[i].values > data['levels_low'].values) & (data[i].values < ((data['levels_low'].values) * mul)), i, np.nan)
#       # data[i] = np.where((data[i].values < data['levels_low'].values) & (data['levels_low'].values  < ((data[i].values) *mul)), i, np.nan)

# for i in range(20,222):
#       # data[i] = np.where((data[i].values > data['levels_high'].values) & (data[i].values < ((data['levels_high'].values) * mul)), i, np.nan)
#       data[i] = np.where((data[i].values < data['levels_high'].values) & (data['levels_high'].values  < ((data[i].values) *mul)), i, np.nan)

  
data=data.drop(['Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'levels', 'levels_low', 'levels_high'], axis=1)
max = data.count().max()
total = pd.DataFrame(data.count())
best = total[total[0]==max].index.values
print(best)




C:\Users\Grant\AppData\Local\Temp\ipykernel_4504\1212021477.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[ema_length] = ta.ema(data["Close"], length=ema_length)


[36 37 38]
